In [1]:
import pandas as pd
import os
from typing import Optional

In [2]:
def calculate_adi(file_path: str) -> Optional[pd.DataFrame]:
    """
    Tính ADI (Average Demand Interval) cho từng cặp (shop_id, item_id).

    ADI = Tổng số ngày trong khoảng thời gian phân tích / Số ngày có nhu cầu (> 0)
    """
    print("--- Bắt đầu tính toán ADI (Average Demand Interval) ---")
    
    try:
        df_sales = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy tệp tại đường dẫn: {file_path}")
        return None

    # 1. Tiền xử lý dữ liệu
    df_sales['date'] = pd.to_datetime(df_sales['date'], format='%d.%m.%Y')

    # Tính tổng khoảng thời gian phân tích (tính theo ngày)
    min_date = df_sales['date'].min()
    max_date = df_sales['date'].max()
    TOTAL_DAYS_SPAN = (max_date - min_date).days + 1
    
    print(f"Khoảng thời gian phân tích: {TOTAL_DAYS_SPAN} ngày.")

    # 2. Lọc nhu cầu thực tế (loại bỏ hàng trả lại hoặc giao dịch bằng 0)
    df_demand = df_sales[df_sales['item_cnt_day'] > 0].copy()

    # 3. Tính số ngày có nhu cầu (> 0) cho mỗi cặp (shop, item)
    # Lấy count của các ngày duy nhất trong mỗi nhóm (shop_id, item_id)
    demand_periods = df_demand.groupby(['shop_id', 'item_id'])['date'].nunique().reset_index(name='demand_periods')

    # 4. Tính ADI
    # ADI = Tổng số ngày / Số ngày có nhu cầu
    demand_periods['ADI'] = TOTAL_DAYS_SPAN / demand_periods['demand_periods']

    # In kết quả thống kê
    print("\n--- Thống kê ADI (Average Demand Interval) ---")
    print(demand_periods['ADI'].describe())
    
    print("\n--- 5 cặp item/shop có ADI cao nhất (nhu cầu không liên tục) ---")
    top_adi = demand_periods.sort_values(by='ADI', ascending=False).head(5)
    
    # --- DÒNG ĐÃ SỬA LỖI ---
    # Thay thế .to_markdown(index=False) bằng print() thông thường
    print(top_adi)

    return demand_periods

In [3]:
# --- Phần thực thi (tương đương với if __name__ == "__main__":) ---
    
# Giả sử tệp notebook (.ipynb) này được lưu trong thư mục 'src/'
# os.getcwd() sẽ trả về thư mục hiện tại (src)
current_dir = os.getcwd()

# Đường dẫn đúng đến tệp sales_train.csv
# (src/data/raw/sales_train.csv)
sales_path = os.path.join(current_dir, 'data', 'raw', 'sales_train.csv')

print(f"Đang phân tích dữ liệu từ: {sales_path}")

if not os.path.exists(sales_path):
    print(f"Lỗi: Không tìm thấy tệp {sales_path}.")
else:
    # Gọi hàm đã được định nghĩa ở ô trên và lưu kết quả
    adi_results = calculate_adi(sales_path)

Đang phân tích dữ liệu từ: c:\Users\user\OneDrive\Máy tính\doantt_KHDL\data_science_project\data_science_ecommerce\src\data\raw\sales_train.csv
--- Bắt đầu tính toán ADI (Average Demand Interval) ---
Khoảng thời gian phân tích: 1034 ngày.

--- Thống kê ADI (Average Demand Interval) ---
count    424098.000000
mean        496.752362
std         385.693388
min           1.192618
25%         147.714286
50%         344.666667
75%        1034.000000
max        1034.000000
Name: ADI, dtype: float64

--- 5 cặp item/shop có ADI cao nhất (nhu cầu không liên tục) ---
        shop_id  item_id  demand_periods     ADI
212049       30    11789               1  1034.0
298804       44    11801               1  1034.0
99617        17     7501               1  1034.0
363073       53     4388               1  1034.0
363067       53     4378               1  1034.0


In [4]:
# Bạn có thể xem lại kết quả (DataFrame) bất cứ lúc nào
if 'adi_results' in locals():
    print("\nXem lại 5 dòng đầu của kết quả ADI:")
    print(adi_results.head())


Xem lại 5 dòng đầu của kết quả ADI:
   shop_id  item_id  demand_periods         ADI
0        0       30               9  114.888889
1        0       31               7  147.714286
2        0       32              11   94.000000
3        0       33               6  172.333333
4        0       35              12   86.166667
